# Create and Run chat flows using Promptflow Python SDK 

### Overview
Prompt flow is a suite of development tools designed to streamline the end-to-end development cycle of LLM-based AI applications, from ideation, prototyping, testing, evaluation to production deployment and monitoring. It makes prompt engineering much easier and enables you to build LLM apps with production quality. 

In this hands-on, you will be able to:
Create flows that link fine-tuned phi3.5 endpoint(Python code) and gpt model in a executable workflow.
Debug and iterate your flows, especially tracing interaction with LLMs with ease.


#### 1. Set up Promptflow client with Credential and configuration
#### 2. Create a new chat flow by providing the flow name and description.
#### 3. Run Basic Promptflow with questions to compare models
#### 4. Run Context Added Promptflow with the outdoor questions 
#### 5. Use serverless endpoint to run the Promptflow with context

[Note] Please use `Python 3.10 - SDK v2 (azureml_py310_sdkv2)` conda environment.


In [3]:
import json
import os
import time

# Import required libraries
from promptflow.azure import PFClient
from promptflow.entities import Run
# Import required libraries
from azure.identity import DefaultAzureCredential, EnvironmentCredential, InteractiveBrowserCredential
from dotenv import load_dotenv
from azure.core.exceptions import HttpResponseError

with open('./config.json', 'r') as f:
    config = json.load(f)
    
print(config["subscription_id"])
print(config["resource_group"])
print(config["workspace_name"]) # Azure AI Studio project name which is not the same as the Azure ML workspace name

your subscription id
your resource group
your workspace name = Azure AI studio project name


In [4]:
from tqdm import tqdm

# Monitor the status of the run_result
def monitor_status(pf_azure_client:PFClient, run_result:Run):
    with tqdm(total=3, desc="Running Status", unit="step") as pbar:
        status = pf_azure_client.runs.get(run_result).status
        if status == "Preparing":
            pbar.update(1)
        while status != "Completed" and status != "Failed":
            if status == "Running" and pbar.n < 2:
                pbar.update(1)
            print(f"Current Status: {status}")
            time.sleep(10)
            status = pf_azure_client.runs.get(run_result).status
        pbar.update(1)
        print("Promptflow Running Completed")

## 1. Set up Promptflow client with Credential and configuration
- Create a promptflow client with the credential and configuration. You need to set the `config.json` file with subscription_id, resource_group and workspace_name

In [6]:
credential = DefaultAzureCredential()

# if you cannot use DefaultAzureCredential, you need to set up the environment variables for the service principal
# credential = ServicePrincipalCredentials(
#     client_id=os.environ["AZURE_CLIENT_ID"],
#     secret=os.environ["AZURE_CLIENT_SECRET"],
#     tenant=os.environ["AZURE_TENANT_ID"],
# )
# Check if given credential can get token successfully.
credential.get_token("https://management.azure.com/.default")

pf_azure_client = PFClient.from_config(credential=credential, path="./config.json")

# pf_azure_client = PFClient(credential=credential, 
#                            subscription_id="your subscription id", 
#                            resource_group_name="your resource group name", 
#                            workspace_name="your workspace name")            

try:
    workspace = pf_azure_client.ml_client.workspaces.get(name=config["workspace_name"])
    print(f"Connected to Azure AI Studio Workspace: {workspace.name}")
    print(f"Workspace Location: {workspace.location}")
    print(f"Workspace ID: {workspace.id}")
except HttpResponseError as e:
    print(f"Failed to connect to Azure ML Workspace: {e}")


Found the config file in: config.json


Failed to connect to Azure ML Workspace: (AuthorizationFailed) The client '8e2870f5-6900-44d5-a7ca-5892562926fd' with object id '8e2870f5-6900-44d5-a7ca-5892562926fd' does not have authorization to perform action 'Microsoft.MachineLearningServices/workspaces/read' over scope '/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourceGroups/slm-innovator-rg/providers/Microsoft.MachineLearningServices/workspaces/your workspace name = Azure AI studio project name' or the scope is invalid. If access was recently granted, please refresh your credentials.
Code: AuthorizationFailed
Message: The client '8e2870f5-6900-44d5-a7ca-5892562926fd' with object id '8e2870f5-6900-44d5-a7ca-5892562926fd' does not have authorization to perform action 'Microsoft.MachineLearningServices/workspaces/read' over scope '/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourceGroups/slm-innovator-rg/providers/Microsoft.MachineLearningServices/workspaces/your workspace name = Azure AI studio project name' or

## Check the exist connections
- currently we only support create connection in Azure AI, ML Studio UI. Check the exiting connections in the workspace.

In [7]:
from promptflow.client import PFClient
from promptflow.entities import AzureOpenAIConnection, CustomConnection

# Get a pf client to get connections
# currently we only support create connection in Azure AI, ML Studio UI
pf = PFClient()
try:
    pf.connections.list()
    for connection in pf.connections.list():
        print(connection)
except (ValueError, TypeError) as e:
    print(e)

auth_mode: key
name: ai-slmhub1616907550322_aoai
module: promptflow.connections
created_date: '2024-10-21 23:44:10.828648+00:00'
last_modified_date: '2024-10-21 23:44:10.828648+00:00'
type: azure_open_ai
api_key: '******'
api_base: https://ai-slmhub1616907550322.openai.azure.com/
api_type: Azure
api_version: 2023-07-01-preview
resource_id: 
  /subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourceGroups/slm-innovator-rg/providers/Microsoft.CognitiveServices/accounts/ai-slmhub1616907550322

auth_mode: key
name: ai-slmhub1616907550322
module: promptflow.connections
created_date: '2024-10-21 23:44:10.828648+00:00'
last_modified_date: '2024-10-21 23:44:10.828648+00:00'
type: azure_ai_services
api_key: '******'
endpoint: https://ai-slmhub1616907550322.cognitiveservices.azure.com/

auth_mode: key
name: wsstq75qvfcwosearch
module: promptflow.connections
created_date: '2024-10-22 08:12:14.702733+00:00'
last_modified_date: '2024-10-22 08:12:14.702733+00:00'
type: cognitive_search
api_key: 

## 2. Create a new chat flow by providing the flow name and description.
- Create a new chat flow by providing the flow name and description. You can view and clone the flow on Azure AI studio UI. 

In [8]:
#Storage Blob Data Contributor, Storage File Data Privileged Contributor role을 managed Identity / service principal / InteractiveBrowserCredential 에게 부여하면 아래 코드를 실행할 수 있습니다.
pf_azure_client.flows.create_or_update(flow="chat/", type="chat", display_name="comparison flow created from python sdk", description="fine-tuned model comparison flow")

Flow created successfully:
{
    "name": "12cc91d9-836d-43bb-83d9-39a2fbcb2482",
    "type": "chat",
    "description": "fine-tuned model comparison flow",
    "path": "Users/8e2870f5-6900-44d5-a7ca-5892562926fd/promptflow/chat-10-25-2024-05-15-04/flow.dag.yaml",
    "code": "azureml://locations/eastus/workspaces/884cbbaf-4d30-486f-a017-f9265971b9e1/flows/12cc91d9-836d-43bb-83d9-39a2fbcb2482",
    "display_name": "comparison flow created from python sdk",
    "owner": {
        "user_object_id": "8e2870f5-6900-44d5-a7ca-5892562926fd",
        "user_tenant_id": "16b3c013-d300-468d-ac64-7eda0820b6d3",
        "user_name": "7ac00d14-31c3-4eac-a9a9-338f6dbd1c98"
    },
    "is_archived": false,
    "created_date": "2024-10-25 05:15:06.718224+00:00",
    "flow_portal_url": "https://ai.azure.com/projectflows/12cc91d9-836d-43bb-83d9-39a2fbcb2482/884cbbaf-4d30-486f-a017-f9265971b9e1/details/Flow?wsid=/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourcegroups/slm-innovator-rg/providers/

## 2. Run Basic Promptflow with questions to compare models
- Run the Promptflow with the simple questions such as "What is the capital of France?" and compare the results of the models

In [23]:
flow_path = "./chat"
data_path = "./data/questions_basic.jsonl"

column_mapping = {
    "question": "${data.question}"
}

run_result = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path,
    column_mapping=column_mapping,
    display_name="chat_with_data",
    tags={"chat_with_jsonl": "", "1st_round": ""},
)


Uploading chat (0.04 MBs): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36734/36734 [00:00<00:00, 492224.22it/s]


Portal url: https://ai.azure.com/projectflows/trace/run/chat_variant_0_20241025_053733_962416/details?wsid=/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourcegroups/slm-innovator-rg/providers/Microsoft.MachineLearningServices/workspaces/slm-pjt1


In [24]:
monitor_status(pf_azure_client, run_result)

Running Status:  33%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 1/3 [00:00<00:01,  1.92step/s]

Current Status: Preparing


Running Status:  67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                                                                                                                                                                                                                                           | 2/3 [00:11<00:06,  6.43s/step]

Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running


Running Status: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:56<00:00, 38.77s/step]

Promptflow Running Completed


In [25]:
detail = pf_azure_client.get_details(run_result)

detail

,inputs.question,inputs.line_number,outputs.phi35_answer,outputs.gpt4o_answer
outputs.line_number,,,,
0,what is the center of Seoul?,0,"The center of Seoul is Gwanghwamun Plaza, whi...",The center of Seoul is often considered to be ...
1,What is the capital of France?,1,The capital of France is Paris. Paris. Paris....,The capital of France is Paris.
2,Tell me about the brief history of Microsoft,2,Microsoft was founded in 1975 by Bill Gates a...,Microsoft was founded by Bill Gates and Paul A...
3,Who wrote Romeo and Juliet?,3,William Shakespeare wrote Romeo and Juliet. I...,William Shakespeare.
4,What does HTML stand for?,4,HTML stands for HyperText Markup Language. It...,HTML stands for HyperText Markup Language.
5,Who painted the Mona Lisa?,5,The Mona Lisa was painted by Leonardo da Vinc...,Leonardo da Vinci.
6,What is the largest planet in our solar system?,6,Jupiter is the largest planet in our solar sy...,Jupiter is the largest planet in our solar sys...
7,Who won the Nobel Prize in Literature in 2024?,7,"As of my knowledge cutoff in March 2023, I do...","I'm sorry, but I don't have information on the..."


## 3. Run Context Added Promptflow with the outdoor questions 
- Run the Promptflow using the context data and ask the outdoor product related questions to compare the results of the models

In [26]:
flow_path = "./chat-context"
data_path = "./data/questions_outdoor.jsonl"

# get the context from context.json file as str and map it to the column_mapping
with open('./data/context_simple.json', 'r') as file:
    context = json.load(file)

column_mapping = {
    "question": "${data.question}",
    "context": context.get("context")    
}

run_result_with_context = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path, 
    column_mapping=column_mapping,
    display_name="chat_context_data",
    tags={"chat_with_context_jsonl": "", "1st_round": ""},
)


Uploading chat-context (0.04 MBs): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36462/36462 [00:00<00:00, 435915.62it/s]


Portal url: https://ai.azure.com/projectflows/trace/run/chat_context_variant_0_20241025_054008_635875/details?wsid=/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourcegroups/slm-innovator-rg/providers/Microsoft.MachineLearningServices/workspaces/slm-pjt1


In [27]:
monitor_status(pf_azure_client, run_result_with_context)

Running Status:   0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0/3 [00:00<?, ?step/s]

Current Status: NotStarted
Current Status: NotStarted
Current Status: NotStarted
Current Status: NotStarted
Current Status: Preparing


Running Status:  33%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 1/3 [00:52<01:44, 52.27s/step]

Current Status: Running


Running Status:  67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                                                                                                                                                                                                                                           | 2/3 [01:02<00:27, 27.71s/step]

Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running


Running Status: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [03:40<00:00, 73.62s/step]

Promptflow Running Completed


In [29]:
detail = pf_azure_client.get_details(run_result_with_context)

detail

,inputs.question,inputs.context,inputs.line_number,outputs.phi35_answer,outputs.gpt4o_answer
outputs.line_number,,,,,
0,tell me about your TrailMaster X4 Tent,TrailMaster X4 Tent is a durable polyester ten...,0,트레일마스터 X4 텐트는 강한 합성 섬유로 만들어진 네 명의 탐험자에게 튼튼한 텐...,"TrailMaster X4 텐트는 4인용으로 설계되어 있으며, 방수 기능이 뛰어나고..."
1,Do you have any climbing gear?,TrailMaster X4 Tent is a durable polyester ten...,1,저는 AI이므로 실시간으로 제품을 보유하지 않습니다. 그러나 등반에 사용하는 컴퍼...,"아쉽지만, 저는 실제 장비는 없어요! 😅 하지만 어떤 장비가 필요한지 도와드릴 수 ..."
2,Can you tell me about your selection of tents?,TrailMaster X4 Tent is a durable polyester ten...,2,None,저희의 텐트 종류에 대해 말씀드릴게요! 🏕️\n\n1. **백패킹 텐트**: 가볍고...
3,Do you have TrekReady Hiking Boots? How much i...,TrailMaster X4 Tent is a durable polyester ten...,3,None,"죄송해요, TrekReady 하이킹 부츠 재고 여부나 가격 정보는 제공할 수 없어요..."
4,can you tell me BaseCamp Folding Table?,TrailMaster X4 Tent is a durable polyester ten...,4,BaseCamp Folding Table는 48x24인치의 가벼운 알루미늄 폴딩 ...,BaseCamp 접이식 테이블은 캠핑이나 야외 활동에 딱 맞는 테이블이에요. 가볍고...


## 4. Use serverless endpoint to run the Promptflow with context
- Create a serverless endpoint to run the Promptflow with the context. You can use the endpoint to run the flow with the context.

### deploy your serverless endpoint

- go to the Azure AI studio > Model catalog > search phi-3.5 > deply Phi-3.5-mini-instruct as your serverless endpint 
<br>
![serverless endpoint](./images/deploy_serverless_endpoint.jpg)
<br>
<br>
- once the deployment is done, go to Deployments and you can see the endpoint deployed in the endpoint section. Click to check the details and copy key and phi35-mini-instruct: Chat Completion endpoint url
![copy connection](./images/copy_connection.jpg)
<br>
<br>
- go to Settings in Azure AI studio > Connections > create a new connection naming phi35-serverless with the copied key and endpoint url
![create new serverless connection](./images/create_new_serverless_connection.jpg)





In [30]:
flow_path = "./chat-serverless"
data_path = "./data/questions_outdoor.jsonl"

# get the context from context.json file as str and map it to the column_mapping
with open('./data/context_simple.json', 'r') as file:
    context = json.load(file)

column_mapping = {
    "question": "${data.question}",
    "context": context.get("context")    
}

run_serverless_result = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path, 
    column_mapping=column_mapping,
    display_name="chat_serverless_context_data",
    tags={"chat_serverless_context_jsonl": "", "1st_round": ""},
)


[2024-10-25 05:47:23 +0000][promptflow][WARNING] - You're using compute session, if it's first time you're using it, it may take a while to build session and you may see 'NotStarted' status for a while. 
[2024-10-25 05:47:23 +0000][promptflow][WARNING] - The trace Cosmos DB for current workspace/project is not ready yet, your traces might not be logged and stored properly.
To enable it, please run `pf config set trace.destination=azureml://subscriptions/<subscription-id>/resourceGroups/<resource-group-name>/providers/Microsoft.MachineLearningServices/workspaces/<workspace-or-project-name>`, prompt flow will help to get everything ready.



Portal url: https://ai.azure.com/projectflows/trace/run/chat_serverless_variant_0_20241025_054722_148223/details?wsid=/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourcegroups/slm-innovator-rg/providers/Microsoft.MachineLearningServices/workspaces/slm-pjt1


In [25]:
monitor_status(pf_azure_client, run_serverless_result)

Running Status:  33%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 1/3 [00:00<00:01,  1.96step/s]

Current Status: Preparing
Current Status: Preparing
Current Status: Preparing


Running Status:  67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                                                                                                                                                                                                                                           | 2/3 [00:32<00:19, 19.11s/step]

Current Status: Running
Current Status: Running
Current Status: Running


Running Status: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:04<00:00, 21.43s/step]

Promptflow Running Completed


In [ ]:
detail = pf_azure_client.get_details(run_serverless_result)

detail